In [1]:
from unlabeled_extrapolation.models import bit_resnet, vit_model, timm_model
from unlabeled_extrapolation.utils import utils
import importlib
import timm
import torch
import numpy as np
importlib.reload(bit_resnet)
importlib.reload(vit_model)
importlib.reload(utils)
importlib.reload(timm_model)

<module 'unlabeled_extrapolation.models.timm_model' from '/juice/scr/ananya/cifar_experiments/transfer_learning/unlabeled_extrapolation/models/timm_model.py'>

In [2]:
import sys
print(sys.path)

['/juice/scr/ananya/cifar_experiments/transfer_learning/scripts', '/juice/scr/ananya/cifar_experiments/transfer_learning/scripts', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python38.zip', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/lib-dynload', '', '/sailhome/ananya/.local/lib/python3.8/site-packages', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/apex-0.1-py3.8-linux-x86_64.egg', '/juice/scr/ananya/cifar_experiments/wilds', '/juice/scr/ananya/cifar_experiments/transfer_learning', '/juice/scr/ananya/verified_calibration', '/u/nlp/anaconda/main/anaconda3/envs/ananya-ue/lib/python3.8/site-packages/IPython/extensions', '/sailhome/ananya/.ipython']


In [2]:
def get_layers_freeze_test(model):
    print('num params before freezing: ', utils.count_parameters(model, trainable=True))
    print(model.get_layers()[1])
    print(len(model.get_layers()))
    for k in [1, 2, len(model.get_layers())]:
        model.freeze_bottom_k(k=k)
        print(f'num params after freezing {k}: {utils.count_parameters(model, trainable=True)}')


In [22]:
# Bit-resnet (get layers) for ResNet-50 and ResNet-101

resnet50_checkpoint_path = "/u/scr/ananya/simclr_weights/BiT-M-R50x1.npz"
resnet50 = bit_resnet.BitResNet(model_name='BiT-M-R50x1', checkpoint_path=resnet50_checkpoint_path)
get_layers_freeze_test(resnet50)

resnet101_checkpoint_path = "/u/scr/ananya/simclr_weights/BiT-M-R101x1.npz"
resnet101 = bit_resnet.BitResNet(model_name='BiT-M-R101x1', checkpoint_path=resnet101_checkpoint_path)
get_layers_freeze_test(resnet101)

num params before freezing:  68256659
('conv-block-0', Sequential(
  (unit01): PreActBottleneck(
    (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (relu): ReLU(inplace=True)
    (downsample): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (unit02): PreActBottleneck(
    (gn1): GroupNorm(32, 256, eps=1e-05, affine=True)
    (conv1): StdConv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
    (conv3)

In [57]:
# DINO
model = vit_model.VitModel(model_name='dino_vitb16')
model.new_last_layer(10)
get_layers_freeze_test(model)


Downloading: "https://github.com/facebookresearch/dino/archive/main.zip" to /sailhome/ananya/.cache/torch/hub/main.zip


num params before freezing:  85806346
('empty_ln_pre', Module())
54
1
num params after freezing 1: 85215754
2
num params after freezing 2: 85215754
54
num params after freezing 54: 0


In [3]:
# Timm ViT-S
model = vit_model.VitModel(model_name='timm.vit_small_patch16_224')
get_layers_freeze_test(model)


num params before freezing:  22050664
('empty_ln_pre', Module())
54
num params after freezing 1: 21755368
num params after freezing 2: 21755368
num params after freezing 54: 0


In [24]:
# Get model grads and norms for Timm ViT-S
layers = model.get_layers()

for name, layer in layers:
    norms_squared = [np.linalg.norm(p.data.detach().cpu().numpy()) ** 2 for p in layer.parameters()]
    norm = np.sqrt(np.sum(norms_squared))
    print(name, norm)

patch_embed 26.6946963982087
empty_ln_pre 0.0
pos_embed 114.1449203491211
cls_token 11.814652442932129
trans0_norm1 10.727939400443407
trans0_attn 51.44874777832921
trans0_norm2 28.292761646663003
trans0_mlp 76.65611477620195
trans1_norm1 11.67158919875855
trans1_attn 51.968497785989186
trans1_norm2 16.081294584238012
trans1_mlp 72.49136824958491
trans2_norm1 14.588461930309668
trans2_attn 47.43609999667546
trans2_norm2 16.750402005493847
trans2_mlp 73.4898046975493
trans3_norm1 17.127796509537827
trans3_attn 46.12080652841503
trans3_norm2 16.560562623359647
trans3_mlp 73.75539281539531
trans4_norm1 16.38613601236467
trans4_attn 45.396057461621595
trans4_norm2 15.56170805432499
trans4_mlp 74.04262678751516
trans5_norm1 15.666596993058972
trans5_attn 45.22580724285582
trans5_norm2 16.125041315399216
trans5_mlp 74.32087040062893
trans6_norm1 16.50691386925591
trans6_attn 44.89644589636653
trans6_norm2 16.870073778570866
trans6_mlp 78.20890488715406
trans7_norm1 16.66077125665377
trans7_a

In [3]:
# Conv-next
model = timm_model.TimmModel('convnext_base_in22k')
get_layers_freeze_test(model)

num params before freezing:  109953489
('stage0', ConvNeXtStage(
  (downsample): Identity()
  (blocks): Sequential(
    (0): ConvNeXtBlock(
      (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
      (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=128, out_features=512, bias=True)
        (act): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=512, out_features=128, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
    )
    (1): ConvNeXtBlock(
      (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
      (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=128, out_features=512, bias=True)
        (act): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=512

In [9]:
x = torch.zeros((8,3,224,224))
x = x.cuda()
model(x)

tensor([[-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        ...,
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625],
        [-2.1558,  0.1969, -2.2230,  ..., -2.2479,  0.7122,  0.2625]],
       device='cuda:0')

In [10]:
print(model._model.head)

Sequential(
  (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=Identity())
  (norm): LayerNorm2d((1024,), eps=1e-06, elementwise_affine=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (drop): Dropout(p=0.0, inplace=False)
  (fc): Linear(in_features=1024, out_features=21841, bias=True)
)
